## Материалы

- https://pendulum.eustace.io/

In [1]:
import pendulum

The default timezone, except when using the `now()`, method will always be UTC.

## Instantiation

`datetime()` sets the time to `00:00:00` if it's not specified, and the timezone (the `tz` keyword argument) to `UTC`.
Otherwise it can be a Timezone instance or simply a string timezone value.

### Без указания tz - UTC

In [2]:
dt = pendulum.datetime(2015, 2, 5)
dt

DateTime(2015, 2, 5, 0, 0, 0, tzinfo=Timezone('UTC'))

In [3]:
dt.timezone, dt.timezone.name, dt.timezone_name

(Timezone('UTC'), 'UTC', 'UTC')

### Явное указание tz

In [4]:
pendulum.datetime(2015, 2, 5, tz='Europe/Paris')

DateTime(2015, 2, 5, 0, 0, 0, tzinfo=Timezone('Europe/Paris'))

In [5]:
tz = pendulum.timezone('Europe/Paris')
pendulum.datetime(2015, 2, 5, tz=tz)

DateTime(2015, 2, 5, 0, 0, 0, tzinfo=Timezone('Europe/Paris'))

Normalization (TODO - разобраться)

In [6]:
pendulum.datetime(2013, 3, 31, 2, 30, tz='Europe/Paris')
# 2:30 for the 31th of March 2013 does not exist
# so pendulum will return the actual time which is 3:30+02:00

DateTime(2013, 3, 31, 3, 30, 0, tzinfo=Timezone('Europe/Paris'))

In [7]:
pendulum.datetime(2013, 3, 31, 2, 30, 0, 0, tz='Europe/Paris', dst_rule=pendulum.PRE_TRANSITION)

DateTime(2013, 3, 31, 1, 30, 0, tzinfo=Timezone('Europe/Paris'))

### local - current timezone

In [8]:
# The special local string is also supported and will return your current timezone.
pendulum.datetime(2015, 2, 5, tz='local')

DateTime(2015, 2, 5, 0, 0, 0, tzinfo=Timezone('Europe/Moscow'))

In [9]:
# local() is just an alias for datetime(..., tz='local')
pendulum.local(2015, 2, 5)

DateTime(2015, 2, 5, 0, 0, 0, tzinfo=Timezone('Europe/Moscow'))

### now, today etc

In [10]:
pendulum.now(), pendulum.now('local'), pendulum.now('Europe/London')

(DateTime(2022, 10, 13, 11, 13, 24, 714891, tzinfo=Timezone('Europe/Moscow')),
 DateTime(2022, 10, 13, 11, 13, 24, 714952, tzinfo=Timezone('Europe/Moscow')),
 DateTime(2022, 10, 13, 9, 13, 24, 714960, tzinfo=Timezone('Europe/London')))

In [11]:
(now_in_london_tz := pendulum.now('Europe/London'))

DateTime(2022, 10, 13, 9, 13, 24, 898040, tzinfo=Timezone('Europe/London'))

In [12]:
now_in_london_tz.timezone_name

'Europe/London'

In [13]:
tt = pendulum.today(), pendulum.yesterday(), pendulum.tomorrow(), pendulum.tomorrow('Europe/London')
tt

(DateTime(2022, 10, 13, 0, 0, 0, tzinfo=Timezone('Europe/Moscow')),
 DateTime(2022, 10, 12, 0, 0, 0, tzinfo=Timezone('Europe/Moscow')),
 DateTime(2022, 10, 14, 0, 0, 0, tzinfo=Timezone('Europe/Moscow')),
 DateTime(2022, 10, 14, 0, 0, 0, tzinfo=Timezone('Europe/London')))

In [14]:
for t in tt: print(t)

2022-10-13T00:00:00+03:00
2022-10-12T00:00:00+03:00
2022-10-14T00:00:00+03:00
2022-10-14T00:00:00+01:00


### naive

Pendulum enforces timezone aware datetimes, and using them is the preferred and recommended way of using the library.
However, if you really need a naive DateTime object, the naive() helper is there for you.

In [15]:
naive = pendulum.naive(2015, 2, 5)
naive, naive.timezone

(DateTime(2015, 2, 5, 0, 0, 0), None)

### from_format()

The next helper, `from_format()`, is similar to the native `datetime.strptime()` function but uses custom tokens to create a `DateTime` instance.

In [16]:
dt = pendulum.from_format('1975-05-21 22', 'YYYY-MM-DD HH')
dt

DateTime(1975, 5, 21, 22, 0, 0, tzinfo=Timezone('UTC'))

In [17]:
# It also accepts a tz keyword argument to specify the timezone
dt = pendulum.from_format('1975-05-21 22', 'YYYY-MM-DD HH', tz='Europe/London')
dt

DateTime(1975, 5, 21, 22, 0, 0, tzinfo=Timezone('Europe/London'))

### from_timestamp()

The final helper is for working with unix timestamps. `from_timestamp()` will create a DateTime instance equal to the given timestamp and will set the timezone as well or default it to UTC.

In [18]:
pendulum.from_timestamp(-1, tz='Europe/London')

DateTime(1970, 1, 1, 0, 59, 59, tzinfo=Timezone('Europe/London'))

## Parsing

The library natively supports the RFC 3339 format, most ISO 8601 formats and some other common formats.

If you pass a non-standard or more complicated string, it will raise an exception, so it is advised to use the `from_format()` helper instead.

However, if you want the library to fall back on the `dateutil` parser, you have to pass `strict=False`.

In [19]:
pendulum.parse('1975-05-21T22:00:00')

DateTime(1975, 5, 21, 22, 0, 0, tzinfo=Timezone('UTC'))

In [20]:
pendulum.parse('1975-05-21T22:00:00', tz='Europe/Paris')

DateTime(1975, 5, 21, 22, 0, 0, tzinfo=Timezone('Europe/Paris'))

In [21]:
# Not ISO 8601 compliant but common
pendulum.parse('1975-05-21 22:00:00')

DateTime(1975, 5, 21, 22, 0, 0, tzinfo=Timezone('UTC'))

If you pass a non-standard or more complicated string, it will raise an exception, so it is advised to use the `from_format()` helper instead.

However, if you want the library to fall back on the `dateutil` parser, you have to pass `strict=False`.

In [22]:
try:
    pendulum.parse('31-01-01')
except Exception as e:
    print(type(e), e)

pendulum.parse('31-01-01', strict=False)

<class 'pendulum.parsing.exceptions.ParserError'> Unable to parse string [31-01-01]


DateTime(2031, 1, 1, 0, 0, 0, tzinfo=Timezone('UTC'))

In [23]:
# You can pass the exact keyword argument to parse() to get the exact type that the string represents:

pendulum.parse('2012-05-03', exact=True)

Date(2012, 5, 3)

## Attributes and Properties

In [24]:
dt = pendulum.parse('2012-09-05T23:26:11.123789')
dt

DateTime(2012, 9, 5, 23, 26, 11, 123789, tzinfo=Timezone('UTC'))

In [25]:
dt.year, dt.month, dt.day, dt.hour, dt.minute, dt.second, dt.microsecond

(2012, 9, 5, 23, 26, 11, 123789)

In [26]:
(
    dt.day_of_week, dt.day_of_year, dt.week_of_month, dt.week_of_year, dt.days_in_month,
    dt.timestamp(), dt.float_timestamp, dt.int_timestamp,
    dt.quarter
)

(3, 249, 2, 36, 30, 1346887571.123789, 1346887571.123789, 1346887571, 3)

In [27]:
pendulum.datetime(1975, 5, 21).age

47

In [28]:
now = pendulum.now()
now

DateTime(2022, 10, 13, 11, 13, 28, 309406, tzinfo=Timezone('Europe/Moscow'))

In [29]:
now.timezone, now.tz,  now.timezone_name

(Timezone('Europe/Moscow'), Timezone('Europe/Moscow'), 'Europe/Moscow')

In [30]:
now.offset, now.offset_hours

(10800, 3.0)

In [31]:
# Indicates if daylight savings time is on
now.is_dst()

False

In [32]:
# Indicates if the instance is in the same timezone as the local timezone
now.is_local()

True

In [33]:
# Indicates if the instance is in the UTC timezonenow
now.is_utc()

False

## Fluent helpers

In [34]:
dt = pendulum.now()
dt.on(1975, 5, 21).at(22, 32, 5).to_datetime_string()

'1975-05-21 22:32:05'

In [35]:
dt.set(tz='Europe/London')

DateTime(2022, 10, 13, 11, 13, 29, 540382, tzinfo=Timezone('Europe/London'))

Setting the timezone just modifies the timezone information without making any conversion, while `in_timezone()` (or `in_tz()`) converts the time in the appropriate timezone.

In [36]:
dt.in_tz('Europe/Paris'), dt.in_timezone('Europe/Paris')

(DateTime(2022, 10, 13, 10, 13, 29, 540382, tzinfo=Timezone('Europe/Paris')),
 DateTime(2022, 10, 13, 10, 13, 29, 540382, tzinfo=Timezone('Europe/Paris')))

## String formatting

The `__str__` magic method is defined to allow `DateTime` instances to be printed as a pretty date string when used in a string context.

The default string representation is the same as the one returned by the isoformat() method.

For localization support see the Localization section.

In [37]:
dt = pendulum.datetime(1975, 12, 25, 14, 15, 16)
dt, repr(dt), str(dt)

(DateTime(1975, 12, 25, 14, 15, 16, tzinfo=Timezone('UTC')),
 "DateTime(1975, 12, 25, 14, 15, 16, tzinfo=Timezone('UTC'))",
 '1975-12-25T14:15:16+00:00')

In [38]:
print(dt)

1975-12-25T14:15:16+00:00


In [39]:
dt.to_date_string()

'1975-12-25'

In [40]:
dt.to_formatted_date_string()

'Dec 25, 1975'

In [41]:
dt.to_time_string()

'14:15:16'

In [42]:
dt.to_datetime_string()

'1975-12-25 14:15:16'

In [43]:
dt.to_day_datetime_string()

'Thu, Dec 25, 1975 2:15 PM'

In [44]:
# You can also use the format() method
dt.format('dddd Do [of] MMMM YYYY HH:mm:ss A')

'Thursday 25th of December 1975 14:15:16 PM'

In [45]:
# Of course, the strftime method is still available
dt.strftime('%I:%M:%S %p')

'02:15:16 PM'

Common Formats

```python
>>> dt = pendulum.now()

>>> dt.to_atom_string()
'1975-12-25T14:15:16-05:00'

>>> dt.to_cookie_string()
'Thursday, 25-Dec-1975 14:15:16 EST'

>>> dt.to_iso8601_string()
'1975-12-25T14:15:16-0500'

>>> dt.to_rfc822_string()
'Thu, 25 Dec 75 14:15:16 -0500'

>>> dt.to_rfc850_string()
'Thursday, 25-Dec-75 14:15:16 EST'

>>> dt.to_rfc1036_string()
'Thu, 25 Dec 75 14:15:16 -0500'

>>> dt.to_rfc1123_string()
'Thu, 25 Dec 1975 14:15:16 -0500'

>>> dt.to_rfc2822_string()
'Thu, 25 Dec 1975 14:15:16 -0500'

>>> dt.to_rfc3339_string()
'1975-12-25T14:15:16-05:00'

>>> dt.to_rss_string()
'Thu, 25 Dec 1975 14:15:16 -0500'

>>> dt.to_w3c_string()
'1975-12-25T14:15:16-05:00'
```

Formatter

Pendulum uses its own formatter when using the format() method.

This format is more intuitive to use than the one used with strftime() and supports more directives.

```
>>> import pendulum

>>> dt = pendulum.datetime(1975, 12, 25, 14, 15, 16)
>>> dt.format('YYYY-MM-DD HH:mm:ss')
'1975-12-25 14:15:16'
```

## Comparison

Simple comparison is offered up via the basic operators. Remember that the comparison is done in the UTC timezone so things aren't always as they seem.

In [46]:
first = pendulum.datetime(2012, 9, 5, 23, 26, 11, 0, tz='America/Toronto')
second = pendulum.datetime(2012, 9, 5, 20, 26, 11, 0, tz='America/Vancouver')
str(first), str(second)

('2012-09-05T23:26:11-04:00', '2012-09-05T20:26:11-07:00')

In [47]:
first == second

True

In [48]:
first = first.on(2012, 1, 1).at(0, 0, 0)
second = second.on(2012, 1, 1).at(0, 0, 0)
# tz is still America/Vancouver for second

first == second, first > second

(False, False)

To handle the most used cases there are some simple helper functions. For the methods that compare to `now()` (ex. `is_today()`) in some manner the `now()` is created in the same timezone as the instance.

In [49]:
dt = pendulum.now()
dt

DateTime(2022, 10, 13, 11, 13, 32, 692771, tzinfo=Timezone('Europe/Moscow'))

In [50]:
dt.is_past(), dt.is_leap_year()

(True, False)

In [51]:
born = pendulum.datetime(1987, 4, 23)
not_birthday = pendulum.datetime(2014, 9, 26)
birthday = pendulum.datetime(2014, 4, 23)

In [52]:
born.is_birthday(not_birthday), born.is_birthday(birthday)

(False, True)

In [53]:
past_birthday = pendulum.now().subtract(years=50)
past_birthday.is_birthday()  # Compares to now by default

True

## Addition and Subtraction

To easily add and subtract time, you can use the `add()` and `subtract()` methods. Each method returns a new `DateTime` instance.

Passing negative values to `add()` is also possible and will act exactly like `subtract()`.

years, months, days, weeks, hours, minutes, seconds

In [54]:
dt = pendulum.datetime(2012, 1, 31)
dt, dt.to_datetime_string()

(DateTime(2012, 1, 31, 0, 0, 0, tzinfo=Timezone('UTC')), '2012-01-31 00:00:00')

In [55]:
dt.add(years=5), dt.subtract(years=1)

(DateTime(2017, 1, 31, 0, 0, 0, tzinfo=Timezone('UTC')),
 DateTime(2011, 1, 31, 0, 0, 0, tzinfo=Timezone('UTC')))

In [56]:

dt1 = dt.add(years=3, months=2, days=6, hours=12, minutes=31, seconds=43)
dt2 = dt.subtract(years=3, months=2, days=6, hours=12, minutes=31, seconds=43)
dt1, dt2

(DateTime(2015, 4, 6, 12, 31, 43, tzinfo=Timezone('UTC')),
 DateTime(2008, 11, 23, 11, 28, 17, tzinfo=Timezone('UTC')))

## Difference

In [57]:
dt_ottawa = pendulum.datetime(2000, 1, 1, tz='America/Toronto')
dt_vancouver = pendulum.datetime(2000, 1, 1, tz='America/Vancouver')

In [58]:
d = dt_ottawa.diff(dt_vancouver)
d, d.in_hours()

(<Period [2000-01-01T00:00:00-05:00 -> 2000-01-01T00:00:00-08:00]>, 3)

In [59]:
dt_ottawa.diff(dt_vancouver, False).in_hours(), dt_vancouver.diff(dt_ottawa, False).in_hours()
# See also in_days(), in_minutes()

(3, -3)

diff_for_humans (see docs)

In [60]:
# pendulum.set_locale('ru')
pendulum.now().diff_for_humans(pendulum.now().subtract(years=1), locale='de')

'1 Jahr später'

## Modifiers

This group of methods performs helpful modifications to a copy of the current instance. You'll notice that the start_of(), next() and previous() methods set the time to 00:00:00 and the end_of() methods set the time to 23:59:59.999999.

The only one slightly different is the average() method. It returns the middle date between itself and the provided DateTime argument.

```python
>>> import pendulum

>>> dt = pendulum.datetime(2012, 1, 31, 12, 0, 0)

>>> dt.start_of('day')
'2012-01-31 00:00:00'

>>> dt.end_of('day')
'2012-01-31 23:59:59'

>>> dt.start_of('month')
'2012-01-01 00:00:00'

>>> dt.end_of('month')
'2012-01-31 23:59:59'

>>> dt.start_of('year')
'2012-01-01 00:00:00'

>>> dt.end_of('year')
'2012-12-31 23:59:59'

>>> dt.start_of('decade')
'2010-01-01 00:00:00'

>>> dt.end_of('decade')
'2019-12-31 23:59:59'

>>> dt.start_of('century')
'2000-01-01 00:00:00'

>>> dt.end_of('century')
'2099-12-31 23:59:59'

>>> dt.start_of('week')
'2012-01-30 00:00:00'
>>> dt.day_of_week == pendulum.MONDAY
True # ISO8601 week starts on Monday

>>> dt.end_of('week')
'2012-02-05 23:59:59'
>>> dt.day_of_week == pendulum.SUNDAY
True # ISO8601 week ends on SUNDAY

>>> dt.next(pendulum.WEDNESDAY)
'2012-02-01 00:00:00'
>>> dt.day_of_week == pendulum.WEDNESDAY
True

>>> dt = pendulum.datetime(2012, 1, 1, 12, 0, 0)
dt.next()
'2012-01-08 00:00:00'
>>> dt.next(keep_time=True)
'2012-01-08T12:00:00+00:00'

>>> dt = pendulum.datetime(2012, 1, 31, 12, 0, 0)
>>> dt.previous(pendulum.WEDNESDAY)
'2012-01-25 00:00:00'
>>> dt.day_of_week == pendulum.WEDNESDAY
True

>>> dt = pendulum.datetime(2012, 1, 1, 12, 0, 0)
>>> dt.previous()
'2011-12-25 00:00:00'
>>> dt.previous(keep_time=True)
'2011-12-25 12:00:00'

>>> start = pendulum.datetime(2014, 1, 1)
>>> end = pendulum.datetime(2014, 1, 30)
>>> start.average(end)
'2014-01-15 12:00:00'

# others that are defined that are similar
# and tha accept month, quarter and year units
# first_of(), last_of(), nth_of()
```

## Timezones

See docs

## Duration

The Duration class is inherited from the native timedelta class. It has many improvements over the base class.

See docs

In [61]:
it = pendulum.duration(days=1177, seconds=7284, microseconds=1234)

In [62]:
it.weeks, it.days

(168, 1177)

In [63]:
it.in_words()

'168 weeks 1 day 2 hours 1 minute 24 seconds'

## Period

See docs

If you want to iterate over a period, you can use the `range()` method:

In [64]:
start = pendulum.datetime(2000, 1, 1)
end = pendulum.datetime(2000, 1, 10)

period = pendulum.period(start, end)

for dt in period.range('days'):
    print(dt)

2000-01-01T00:00:00+00:00
2000-01-02T00:00:00+00:00
2000-01-03T00:00:00+00:00
2000-01-04T00:00:00+00:00
2000-01-05T00:00:00+00:00
2000-01-06T00:00:00+00:00
2000-01-07T00:00:00+00:00
2000-01-08T00:00:00+00:00
2000-01-09T00:00:00+00:00
2000-01-10T00:00:00+00:00


Supported units for range() are: years, months, weeks, days, hours, minutes and seconds.

In [65]:
dt = pendulum.datetime(2000, 1, 4)
dt in period

True

## График прививок

In [66]:
zero_day = pendulum.from_format('7.08.2022', 'DD.MM.YYYY')
str(zero_day)

'2022-08-07T00:00:00+00:00'

In [67]:
# zero_day = pendulum.today(tz='UTC')

In [68]:
[f'{d} day - {zero_day.add(days=d).format("DD.MM.YYYY")}' for d in [0, 3, 7, 14, 30, 90]]

['0 day - 07.08.2022',
 '3 day - 10.08.2022',
 '7 day - 14.08.2022',
 '14 day - 21.08.2022',
 '30 day - 06.09.2022',
 '90 day - 05.11.2022']